In [1]:
import os
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# 获取当前目录
current_dir = os.getcwd()

file_list = []
# 遍历当前目录及其子目录
for root, dirs, files in os.walk(current_dir):
    for file in files:
        if file.endswith(".mat"):
           file_list.append(file)

In [3]:
file_list.sort()
file_list

['CONE_D1_C_Fea_keli_4_hh.mat',
 'CONE_D1_C_Fea_keli_4_vv.mat',
 'CONE_D1_S_Fea_keli_4_hh.mat',
 'CONE_D1_S_Fea_keli_4_vv.mat',
 'CONE_D1_X_Fea_keli_4_hh.mat',
 'CONE_D1_X_Fea_keli_4_vv.mat',
 'CONE_D2_C_Fea_keli_4_hh.mat',
 'CONE_D2_C_Fea_keli_4_vv.mat',
 'CONE_D2_S_Fea_keli_4_hh.mat',
 'CONE_D2_S_Fea_keli_4_vv.mat',
 'CONE_D2_X_Fea_keli_4_hh.mat',
 'CONE_D2_X_Fea_keli_4_vv.mat',
 'CONE_D3_C_Fea_keli_4_hh.mat',
 'CONE_D3_C_Fea_keli_4_vv.mat',
 'CONE_D3_S_Fea_keli_4_hh.mat',
 'CONE_D3_S_Fea_keli_4_vv.mat',
 'CONE_D3_X_Fea_keli_4_hh.mat',
 'CONE_D3_X_Fea_keli_4_vv.mat',
 'CONE_D4_C_Fea_keli_4_hh.mat',
 'CONE_D4_C_Fea_keli_4_vv.mat',
 'CONE_D4_S_Fea_keli_4_hh.mat',
 'CONE_D4_S_Fea_keli_4_vv.mat',
 'CONE_D4_X_Fea_keli_4_hh.mat',
 'CONE_D4_X_Fea_keli_4_vv.mat',
 'CONE_D5_C_Fea_keli_4_hh.mat',
 'CONE_D5_C_Fea_keli_4_vv.mat',
 'CONE_D5_S_Fea_keli_4_hh.mat',
 'CONE_D5_S_Fea_keli_4_vv.mat',
 'CONE_D5_X_Fea_keli_4_hh.mat',
 'CONE_D5_X_Fea_keli_4_vv.mat',
 'CONE_L1_C_Fea_keli_4_hh.mat',
 'CONE_L

In [4]:
L_fix = [0.03, 1]
D_fix = [4, 0.03]
R_fix = [4, 1]

In [5]:
L_change = [1, 1.3, 1.8, 2.3, 2.8, 3.3, 3.8, 4.3, 4.8]
D_change = [0.5, 0.8, 1.3, 1.8, 2.3]
R_change = [0.005, 0.02, 0.045, 0.07, 0.095]

In [6]:
angle = np.linspace(0, 180, 91)

In [7]:
L_param = np.array([L_fix+[item] for item in [item for item in L_change for _ in range(2)]])
D_param = np.array([[item] + D_fix for item in [item for item in D_change for _ in range(2)]])
R_param = np.array([[R_fix[0]]+[item]+[R_fix[1]] for item in [item for item in R_change for _ in range(2)]])

L_param = np.vstack([L_param, L_param, L_param])
D_param = np.vstack([D_param, D_param, D_param])
R_param = np.vstack([R_param, R_param, R_param])

In [8]:
print(L_param.shape, D_param.shape, R_param.shape)

(54, 3) (30, 3) (30, 3)


In [9]:
# 要复制的份数
k = len(angle)


# 初始化结果列表
L = []

# 遍历原数组中的每一行
for row in L_param:
    # 遍历每一个要添加的元素
    for element in angle:
        # 将原行与新元素结合，并添加到结果中
        new_row = np.append(row, element)
        L.append(new_row)

L = np.array(L)

# 初始化结果列表
D = []

# 遍历原数组中的每一行
for row in D_param:
    # 遍历每一个要添加的元素
    for element in angle:
        # 将原行与新元素结合，并添加到结果中
        new_row = np.append(row, element)
        D.append(new_row)

D = np.array(D)

# 初始化结果列表
R = []

# 遍历原数组中的每一行
for row in R_param:
    # 遍历每一个要添加的元素
    for element in angle:
        # 将原行与新元素结合，并添加到结果中
        new_row = np.append(row, element)
        R.append(new_row)

R = np.array(R)

In [10]:
print(L.shape, D.shape, R.shape)

(4914, 4) (2730, 4) (2730, 4)


In [57]:
y_list = []

for file in file_list:
    data = scipy.io.loadmat(file)
    mean = data['fea_jing'][0][0][0][0][0][7].reshape(-1,)
    y_list.append(mean)

total_y = np.array(y_list).reshape(114*91, 1)

In [58]:
total_y.shape

(10374, 1)

In [59]:
X_feature = np.vstack([D, L, R])
X_feature.shape

(10374, 4)

In [60]:
y_feature = total_y.reshape(-1,)

In [61]:
# 分割数据集
# X_train, X_test, y_train, y_test = train_test_split(X_feature, y_feature, test_size=0.2, random_state=42)

X_train = X_feature[:90*91]
X_test = X_feature[90*91:]
y_train = y_feature[:90*91]
y_test = y_feature[90*91:]

In [62]:
# 转换为PyTorch的张量
x_tensor = torch.from_numpy(X_train)
y_tensor = torch.from_numpy(y_train)



In [63]:
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 创建模型实例
input_dim = X_train.shape[1]
model = RegressionModel(input_dim)


In [64]:
# 将数据转换为PyTorch张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# 创建数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 400
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        # 前向传播
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/400], Loss: 99.7968
Epoch [20/400], Loss: 51.1290
Epoch [30/400], Loss: 64.3130
Epoch [40/400], Loss: 79.5267
Epoch [50/400], Loss: 52.3329
Epoch [60/400], Loss: 60.7901
Epoch [70/400], Loss: 40.1535
Epoch [80/400], Loss: 68.6038
Epoch [90/400], Loss: 67.1484
Epoch [100/400], Loss: 71.4822
Epoch [110/400], Loss: 66.4576
Epoch [120/400], Loss: 75.4805
Epoch [130/400], Loss: 63.6834
Epoch [140/400], Loss: 46.6768
Epoch [150/400], Loss: 48.4891
Epoch [160/400], Loss: 61.4897
Epoch [170/400], Loss: 40.6355
Epoch [180/400], Loss: 69.2806
Epoch [190/400], Loss: 35.3045
Epoch [200/400], Loss: 37.9887
Epoch [210/400], Loss: 28.1799
Epoch [220/400], Loss: 27.5051
Epoch [230/400], Loss: 37.7595
Epoch [240/400], Loss: 42.1170
Epoch [250/400], Loss: 25.6358
Epoch [260/400], Loss: 26.6165
Epoch [270/400], Loss: 27.0211
Epoch [280/400], Loss: 21.5024
Epoch [290/400], Loss: 35.4507
Epoch [300/400], Loss: 28.0108
Epoch [310/400], Loss: 26.1838
Epoch [320/400], Loss: 28.5474
Epoch [330/400], 

In [65]:
# 模型评估
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 50.1708


In [66]:
predict_first = model(torch.tensor(X_feature, dtype=torch.float32)[-182:-91]).detach().numpy()
first = y_feature[-182:-91]

# 可视化结果
plt.plot(first, 'r-', label='Original data')
plt.plot(predict_first, 'b-', label='Fitted line')
plt.legend()
plt.show()